# Final Data Combining and Cleaning

In this notebook, the data is queried, combined, and cleaned. Additionally, the resulting DataFrame is assessed for missing data.

### Final data:
- Temporal resolution: hourly
- Spatial resolution: California Independent System Operator (CAISO) system-wide
>- All data was available on CAISO-wide resolution except for locational marginal price (LMP) data, representing electricity price at specific grid locations
>- LMP data was aggregated to the system-wide level as a load-weighted sum of default load aggregation point (DLAP) LMPs; DLAPs are nodes where regional demand is aggregated and priced
>- All primary CAISO DLAPs are included in this analysis: PGE, SCE, SDGE, and VEA

### Data Sources

All data was pulled from CAISO sources, described below:

- Production, curtailment, and imports/exports: https://www.caiso.com/library/production-curtailments-data
- Load: https://www.caiso.com/library/historical-ems-hourly-load
>- Pre-cleaning performed prior to import into Jupyter notebook: Removed 'CAISO Public' cell from the bottom of some of the CSVs (not present on every CSV) to prevent issues when reading into a DataFrame
- LMP and LMP congestion: https://oasis.caiso.com/mrioasis/logon.do
>- Pre-cleaning performed prior to import into Jupyter notebook: see 'LMP_data_query.ipynb' for data query and modification
- Battery storage: https://www.caiso.com/library/daily-energy-storage-reports
>- Pre-cleaning performed prior to import into Jupyter notebook: CAISO releases battery storage reports quarterly in .XLS format ('storage-report-2023q1', etc.). The data from the four 2023 and four 2024 reports' 'market_output' tab was copied and pasted into a single Excel file. Then Excel filtering was used to only retain data for 'RES_TYPE' = 'LESR' (limited-energy storage resources in CAISO) and 'MARKET' = 'RTD' (the real-time market). The 'TYPE' column was used to filter for 'EN' which represents charging deltas in an interval and 'SOC' which represents the state-of-charge at the end of an interval. The data for 'SOC' and 'EN' was separated into two separate Excel files, 'battery_soc' and 'battery_en'. Finally, the battery data was aggregated hourly. The original data includes columns for hour (1-24) and interval (1-12, every 5 minutes). For 'SOC', the hourly value was calculated as the average of the values for the 12 intervals. For 'EN', the hourly value was taken as the 12th interval's value from each hour. Once the hourly value was selected, the interval column was removed.

In [1]:
## Import required libraries
import pandas as pd
import gridstatus
import os
import matplotlib.pyplot as plt
from scipy.stats import normaltest
from scipy.stats import median_test

Importing and cleaning production, curtailment, export, load, battery storage, and LMP data from 2023 & 2024 to compile a single tidy DataFrame.

## Production, Curtailment, and Import/Export Data

First, we import the Excel files that include total generation and generation by source from the 'Production' tab of the production and curtailments Excel files.

In [2]:
## Import, format, and aggregate production and import/export data
prod_df = pd.DataFrame()

## Compile data from 2023 & 2024
for year in ['2023', '2024']:
    prod = pd.DataFrame()
    
    # Production data
    prod = pd.read_excel(fr"Production_Curtailment/productionandcurtailmentsdata_{year}.xlsx", sheet_name='Production')
    prod['timestamp'] = pd.to_datetime(prod['Date']) # 'Date' column already includes time
    prod['timestamp'] = prod['timestamp'].dt.tz_localize('US/Pacific', ambiguous='infer', nonexistent='shift_forward').dt.tz_convert('UTC') # Handle daylight savings and convert to UTC
    print(f'Duplicate timestamps for production data in {year}:', prod['timestamp'].duplicated().sum()) # Check for duplicated timestamps after converting to UTC
    
    prod.drop(columns=['Date', 'Hour', 'Interval'], inplace=True) # Drop columns that are problematic for aggregation
    prod = prod.set_index('timestamp').resample('h').sum().reset_index() # Aggregate total production by hour
    prod['percent_wind_gen'] = prod['Wind'] / prod['Generation'] # New column for % wind generation
    prod['percent_solar_gen'] = prod['Solar'] / prod['Generation'] # New column for % solar generation

    # Concatenate to existing data
    prod_df = pd.concat([prod_df, prod], ignore_index=True)

Duplicate timestamps for production data in 2023: 0
Duplicate timestamps for production data in 2024: 0


In [3]:
# Check legnth of production data compared to total number of hours in the range
print('Total hours in range: ', 365 * 24 + 366 * 24) # 2024 is a leap year
print('Total hours in production data: ', len(prod_df))

Total hours in range:  17544
Total hours in production data:  17544


Next, we import the Excel files that include curtailment by type (wind or solar) from the 'Curtailments' tab of the production and curtailments Excel files.

In [4]:
## Import, format, and aggregate curtailment data
curt_df = pd.DataFrame()

## Compile data from 2023 & 2024
for year in ['2023', '2024']:
    curt = pd.DataFrame()

    # Curtailment data
    curt = pd.read_excel(fr"Production_Curtailment/productionandcurtailmentsdata_{year}.xlsx", sheet_name='Curtailments')
    curt = curt.groupby(['Date', 'Hour', 'Interval']).agg({'Wind Curtailment': 'sum', 'Solar Curtailment': 'sum'}).reset_index() # Aggregate across reasons 'System' and 'Local' before adding timestamps
    curt['timestamp'] = pd.to_datetime(curt['Date']) + pd.to_timedelta(curt['Hour'] - 1, unit='h') # Create timestamp 'Day' and'Hour' columns
    
    curt['timestamp'] = curt['timestamp'].dt.tz_localize('US/Pacific', ambiguous='NaT', nonexistent='shift_forward') # Localize to US/Pacific time - cannot infer because timestamps are sparse
    curt['timestamp'] = curt['timestamp'].dt.tz_convert('UTC') # Convert to UTC

    curt.drop(columns=['Date', 'Hour', 'Interval'], inplace=True) # Drop columns that are problematic for aggregation
    curt = curt.groupby(curt['timestamp']).sum().reset_index()
    print(f'Duplicate timestamps for curtailment data in {year}:', curt['timestamp'].duplicated().sum()) # Check for duplicated timestamps

    curt_df = pd.concat([curt_df, curt], ignore_index=True)

Duplicate timestamps for curtailment data in 2023: 0
Duplicate timestamps for curtailment data in 2024: 0


In [5]:
# Check legnth of curtailment data compared to total number of hours in the range
print('Total hours in range: ', 365 * 24 + 366 * 24) # 2024 is a leap year
print('Total hours in curtailment data: ', len(curt_df))

Total hours in range:  17544
Total hours in curtailment data:  9298


The curtailment DataFrame does not include values for every hour in the timerange because curtailment does not not occur every interval. This is stated in the 'read_me' tab of the source Excel files.

Now, we can merge our production and curtailment data using our timestamps.

In [6]:
## Merge production and curtailment data using production timestamps
prod_and_curt = pd.merge(prod_df, curt_df, left_on=['timestamp'], right_on=['timestamp'], how='left') # Merge on timestamp

# Filter only for important columns
prod_and_curt = prod_and_curt[['timestamp', 'percent_wind_gen', 'percent_solar_gen', 'Wind Curtailment', 'Solar Curtailment', 'Imports']]
prod_and_curt['total_curtailment'] = prod_and_curt['Wind Curtailment'] + prod_and_curt['Solar Curtailment'] # New column for total curtailment
prod_and_curt['exports'] = -1 * prod_and_curt['Imports'] # Change sign to represent exports instead of imports

# Fill in intervals with no curtailment with 0
prod_and_curt['total_curtailment'] = prod_and_curt['total_curtailment'].fillna(0) # Not every hour has any curtailment, so fill in NA with zero

In [7]:
## Rename and drop columns
prod_and_curt.drop(['Wind Curtailment', 'Solar Curtailment', 'Imports'], axis=1, inplace=True)

## Load Data

Next, we will load our demand data. Demand data was released in a singular Excel report for 2023, but individual monthly Excel reports were released for 2024. We will loop through that folder to pull all of the data for the year.

In [8]:
## Import, format, and aggregate load data from 2023 and 2024

## 2024 load data is separated by month -> pull each file from folder and concatenate
folder = r"Load/2024"
all_files = [f for f in os.listdir(folder) if f.endswith('.xlsx')]

load_2024 = pd.DataFrame()

for file in all_files: # Each file in folder
    path = os.path.join(folder, file)

    load = pd.read_excel(path) # Read file
    load_2024 = pd.concat([load_2024, load], ignore_index=True) # Concatenate to existing
    load_2024 = load_2024.sort_values(['Date', 'HR']).reset_index(drop=True) # Sort by time

## Pull 2023 data - all in one file
load_df = pd.read_excel(r"Load/historicalemshourlyloadfor2023.xlsx")

## Combine into one DataFrame
load_df = pd.concat([load_df, load_2024], ignore_index=True)
load_df = load_df.sort_values(['Date', 'HR']).reset_index(drop=True) # Sort by time

In [9]:
## Convert day/time columns to timestamps in load data
load_df['timestamp'] = pd.to_datetime(load_df['Date']) + pd.to_timedelta(load_df['HR'], unit='h') # Create timestamp column from 'Date' and 'Hour'; Hour 1 is actually 1:00 - 1:59 in this data based on daylight savings
load_df['timestamp'] = load_df['timestamp'].dt.tz_localize('US/Pacific', ambiguous='infer', nonexistent='shift_forward') # Convert to Pacific and handle daylight savings time; continuous, so ambiguous = 'infer' should be OK
print('Duplicate timestamps for load data:', load_df['timestamp'].duplicated().sum()) 

load_df.rename(columns={'CAISO': 'total_load'}, inplace=True) # Rename total load column

Duplicate timestamps for load data: 0


## Locational Marginal Price Data

Total LMP data for 2023-2024 was pulled from the CAISO OASIS API in the 'LMP_data_query.ipynb' notebook. It is already UTC-formatted.

In [10]:
## Import LMP data generated by other notebook
lmp_df = pd.read_parquet('LMP/LMP_data.parquet').reset_index() # non-timestamp index easier for merging/operations

We will calculate a system-wide LMP using a load-weighted sum of the DLAP-specific LMPs.

In [11]:
## Calculate system-wide DLAP load-weighted LMP using load and LMP data
system_lmp_df = pd.DataFrame()
system_lmp_df['total_lmp'] = (load_df['PGE'] * lmp_df['DLAP_PGAE-APND'] + load_df['SCE'] * lmp_df['DLAP_SCE-APND'] + load_df['SDGE'] * lmp_df['DLAP_SDGE-APND'] + load_df['VEA'] * lmp_df['DLAP_VEA-APND']) / load_df['total_load']
system_lmp_df.reset_index(inplace=True)

## Battery Storage Data

We will use the battery charging delta (EN) and battery state of charge (SOC) data post pre-cleaning (described in the header).

In [12]:
## Import, format, and aggregate battery charging delta data
battery_en_df = pd.read_csv(r'Battery/battery_en.csv')

## Fix time alignment in battery storage data - spring forward is normal, but instead of duplicating an hour for fall back, hours count from 1 - 25
## Fixing this manually by shifting the spring forward hours back before localizing and generating timestamps
fall_back_2023 = '2023-11-05'
fall_back_2024 = '2024-11-03'

mask_2023 = battery_en_df['TRADE_DATE'] == fall_back_2023
battery_en_df.loc[mask_2023 & (battery_en_df['HOUR'] >= 3), 'HOUR'] -= 1

mask_2024 = battery_en_df['TRADE_DATE'] == fall_back_2024
battery_en_df.loc[mask_2024 & (battery_en_df['HOUR'] >= 3), 'HOUR'] -= 1

## Now create timestamp, localize, and convert to UTC
battery_en_df['timestamp'] = pd.to_datetime(battery_en_df['TRADE_DATE']) + pd.to_timedelta(battery_en_df['HOUR'] - 1, unit='h') # Create timestamp column from 'TRADE_DATE' and 'HOUR'
battery_en_df['timestamp'] = pd.to_datetime(battery_en_df['timestamp']).dt.tz_localize('US/Pacific', ambiguous='infer', nonexistent='shift_forward').dt.tz_convert('UTC') # Convert to Pacific and handle daylight savings time and then convert to UTC
print('Duplicate timestamps for battery charging delta data:', battery_en_df['timestamp'].duplicated().sum()) # Check for duplicate timestamps after aligning to UTC

## Rename columns
battery_en_df.rename(columns={'VALUE': 'battery_en'}, inplace=True)

Duplicate timestamps for battery charging delta data: 0


In [13]:
## Import, format, and aggregate battery state-of-charge data
battery_soc_df = pd.read_csv(r'Battery/battery_soc.csv')

## Fix time alignment in battery storage data - spring forward is normal, but instead of duplicating an hour for fall back, hours count from 1 - 25
## Fixing this manually by shifting the spring forward hours back before localizing and generating timestamps
mask_2023 = battery_soc_df['TRADE_DATE'] == fall_back_2023
battery_soc_df.loc[mask_2023 & (battery_soc_df['HOUR'] >= 3), 'HOUR'] -= 1

mask_2024 = battery_soc_df['TRADE_DATE'] == fall_back_2024
battery_soc_df.loc[mask_2024 & (battery_soc_df['HOUR'] >= 3), 'HOUR'] -= 1

## Now create timestamp, localize, and convert to UTC
battery_soc_df['timestamp'] = pd.to_datetime(battery_soc_df['TRADE_DATE']) + pd.to_timedelta(battery_soc_df['HOUR'] - 1, unit='h') # Create timestamp column from 'TRADE_DATE' and 'HOUR'
battery_soc_df['timestamp'] = pd.to_datetime(battery_soc_df['timestamp']).dt.tz_localize('US/Pacific', ambiguous='infer', nonexistent='shift_forward').dt.tz_convert('UTC') # Convert to Pacific and handle daylight savings time and then convert to UTC

## Rename columns
battery_soc_df.rename(columns={'VALUE': 'battery_soc'}, inplace=True)

In [14]:
## Check lengths of datasets
print('Hours in time range of interest (2023-2024): ', 24*365*2+24) # 2024 was a leap year
print('Length of production and curtailment data: ', len(prod_and_curt))
print('Length of load data: ', len(load_df))
print('Length of LMP data: ', len(lmp_df))
print('Length of battery charing delta data: ', len(battery_en_df))
print('Length of battery state-of-charge data: ', len(battery_soc_df))

Hours in time range of interest (2023-2024):  17544
Length of production and curtailment data:  17544
Length of load data:  17544
Length of LMP data:  17544
Length of battery charing delta data:  17497
Length of battery state-of-charge data:  17497


The battery datasets are not the same length as the other DataFrames, so they will need to be merged by timestamp. Otherwise, the data could become time-misaligned. The other datasets can be merged on index. We start by merging the production/curtailment, load, and LMP data by index. Then we merge the battery data by timestamp.

In [15]:
## Create final DataFrame to merge all data sources together
data_df = pd.DataFrame()
data_df = pd.merge(prod_and_curt, load_df[['total_load']], left_index=True, right_index=True, how='outer') # Merge production, curtailment, imports/exports, and load
data_df = pd.merge(data_df, system_lmp_df[['total_lmp']], left_index=True, right_index=True, how='outer') # Merge in LMP and congestion LMP

In [16]:
## Merge in battery storage by timestamps because 47 hours are missing
data_df = pd.merge(data_df, battery_en_df[['timestamp', 'battery_en']], on='timestamp', how='left') # Merge left to ensure all hours are kept
data_df = pd.merge(data_df, battery_soc_df[['timestamp', 'battery_soc']], on='timestamp', how='left') # Merge left to ensure all hours are kept

We next extract the month from our timestamp to convert into dummy season variables.

In [17]:
## Set timestamp as index and add columns for hour and month
data_df.set_index('timestamp', inplace=True)
data_df['month'] = data_df.index.month

In [18]:
## Convert months into categories (season) for broader temporal patterns
season_mapping = {12: 'winter', 1: 'winter', 2: 'winter',
                  3: 'spring', 4: 'spring', 5: 'spring',
                  6: 'summer', 7: 'summer', 8: 'summer',
                  9: 'fall', 10: 'fall', 11: 'fall'}

data_df['season'] = data_df['month'].map(season_mapping)

# Dummy variables for seasons
data_df = pd.get_dummies(data_df, columns=['season'], drop_first=True)

# Convert to integers
dummy_cols = ['season_summer', 'season_winter', 'season_spring']
data_df[dummy_cols] = data_df[dummy_cols].astype(int)

In [19]:
## Check for missing values
print('Missing data values by column: ')
print(data_df.isna().sum())

Missing data values by column: 
percent_wind_gen      0
percent_solar_gen     0
total_curtailment     0
exports               0
total_load            0
total_lmp             0
battery_en           47
battery_soc          47
month                 0
season_spring         0
season_summer         0
season_winter         0
dtype: int64


As we saw before, both battery features are missing 47 hours of data. We will determine the importance of this missing data.

## Missing Data Assessment

Check whether 47 missing hours of battery storage data are significant.

In [20]:
## Identify how much data is missing
missing_data = data_df[data_df['battery_en'].isnull() | data_df['battery_soc'].isnull()]
print(f'{round((len(missing_data)/len(data_df))*100, 3)}% of hours missing battery storage information.')

0.268% of hours missing battery storage information.


This is a tiny fraction of our overall data, so we can fill it in linearly.

In [21]:
## Fill in missing battery data linearly
data_df['battery_soc'] = data_df['battery_soc'].interpolate(method='linear')

# Use changes in storage data to fill in missing charge/discharge data
data_df['storage_delta'] = data_df['battery_soc'].diff() # Capture changes in battery storage
data_df['battery_en'] = data_df['battery_en'].fillna(data_df['storage_delta']) # Fill missing values in charging/discharging data
data_df = data_df.drop(columns='storage_delta') # Drop storage delta data from DataFrame

In [22]:
## Validate that there are no more missing values
print('Missing data values by column: ')
print(data_df.isna().sum())

Missing data values by column: 
percent_wind_gen     0
percent_solar_gen    0
total_curtailment    0
exports              0
total_load           0
total_lmp            0
battery_en           0
battery_soc          0
month                0
season_spring        0
season_summer        0
season_winter        0
dtype: int64


Now, we can reorganize our columns and drop our timestamp. Finally, we can save our data to a CSV.

In [23]:
## Reorder columns to have response variable at end and drop timestamp (not actually a feature)
data_df = data_df[['percent_wind_gen', 'percent_solar_gen', 'exports', 'total_load', 'total_lmp', 'battery_en', 'battery_soc', 'season_summer', 'season_spring', 'season_winter', 'total_curtailment']]
data_df.reset_index(inplace=True)
data_df.drop('timestamp', axis=1, inplace=True)

In [24]:
## Preview the data
data_df.head(n=10)

,percent_wind_gen,percent_solar_gen,exports,total_load,total_lmp,battery_en,battery_soc,season_summer,season_spring,season_winter,total_curtailment
0,0.265874,0.000000,-69960.360000,21302.162944,114.687870,41.976,6524.477167,0,0,1,0.000
1,0.303146,0.000000,-67462.573333,20803.750237,108.752210,-5.863,6460.896417,0,0,1,0.000
2,0.327688,0.000000,-65642.693333,20300.078086,102.776174,-54.224,6465.033667,0,0,1,0.000
3,0.326433,0.000000,-64974.106667,19914.921017,106.676475,19.609,6460.216500,0,0,1,0.000
4,0.292937,0.000000,-65427.986667,19975.842246,110.358796,8.992,6428.138083,0,0,1,0.000
5,0.279927,0.000000,-61790.453333,20273.688572,112.348333,581.266,6564.830417,0,0,1,0.000
6,0.274556,0.000000,-61654.080000,20534.584181,113.756832,1112.432,6379.488417,0,0,1,0.000
7,0.254857,0.034031,-51741.000000,20510.883304,103.788766,1043.191,5441.492833,0,0,1,0.000
8,0.240082,0.216341,-38545.640000,20179.969086,112.740669,6.937,5108.654417,0,0,1,0.000
9,0.224064,0.324019,-20055.786667,19656.011666,103.003784,-791.843,5420.171250,0,0,1,2.925


In [25]:
## Save data as CSV
data_df.to_csv('cleaned_data.csv', index=True) 